In [97]:
import os 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 
import math
from deep_translator import GoogleTranslator

In [98]:
def summarize_lexRank(sentences,tokens,threshold=0.1,epsilon=0.1):
    freq_matrix = {}
    ts=len(tokens)
    for i,token in enumerate(tokens):
        freq_table = {}
        words = token
        for word in words:
            word = word.lower()
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        freq_matrix[i] = freq_table
    tf_matrix = {}
    c = 0
    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[c] = tf_table
        c = c+1
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1
    idf_matrix = {}
    idf_table = {}
    for sent, f_table in freq_matrix.items():

        for word in f_table.keys():
          if word not in idf_table:
            idf_table[word] = math.log10(ts / float(word_per_doc_table[word]))
          else:
            pass

        idf_matrix[sent] = idf_table
    matrix = np.zeros((ts, ts))
    s = {}
    for i in range(ts):
      for j in range(ts):
        u1 = tokens[i]
        u2 = tokens[j]
        common = list(set(u1) & set(u2))
        d1=0
        d2=0
        n=0.0
        for t in u1:
          if t in tf_matrix[i]:
            tf1=tf_matrix[i][t]
          else:
            tf1=0
          if t in idf_table:
            idf1=idf_table[t]
          else:
            idf1=0
          d1+=(tf1*idf1)**2
        for t in u2:
          if t in tf_matrix[j]:
            tf2=tf_matrix[j][t]
          else:
            tf2=0
          if t in idf_table:
            idf2=idf_table[t]
          else:
            idf2=0
          d2+=(tf2*idf2)**2
        for t in common:
          if t in tf_matrix[i]:
            tfc1=tf_matrix[i][t]
          else:
            tfc1=0
          if t in tf_matrix[j]:
            tfc2=tf_matrix[j][t]
          else:
            tfc2=0
          if t in idf_table:
            idf=idf_table[t]
          else:
            idf=0
          n+=tfc1*tfc2*idf**2
        if d1 > 0 and d2 > 0:
          matrix[i][j] =  n / (math.sqrt(d1) * math.sqrt(d2))
        else:
          matrix[i][j] = 0.0
    degrees = np.zeros((ts, ))
    for i in range(ts):
      for j in range(ts):
        if matrix[i, j] > threshold:
          matrix[i, j] = 1.0
          degrees[i] += 1
        else:
          matrix[i, j] = 0
    for i in range(ts):
        for j in range(ts):
            if degrees[i] == 0:
                degrees[i] = 1

            matrix[i][j] = matrix[i][j] / degrees[i]
    transposed_matrix = matrix.T
    p_vector = np.array([1.0 / ts] * ts)
    lambda_val = 1.0

    while lambda_val > epsilon:
      next_p = np.dot(transposed_matrix, p_vector)
      lambda_val = np.linalg.norm(np.subtract(next_p, p_vector))
      p_vector = next_p
    avg = np.sum(p_vector) / len(p_vector)
    sentence_ids = []
    for i in range(ts):
      if(p_vector[i]>=avg):
        sentence_ids.append(i)
    summary = [sentences[i] for i in sentence_ids]
    return summary

In [99]:
def split_long_sentence(sentence, max_length=3000):
    parts = []
    for i in range(0, len(sentence), max_length):
        part = sentence[i:i + max_length]
        parts.append(part)

    return parts

def convertToHindi(sentences):
    hindi = []
    for sentence_array in sentences:
        sentence = ''
        hindiText=''
        for i in range(len(sentence_array)):
            sentence = sentence + sentence_array[i]
        if len(sentence)>2000:
            small_sentences = split_long_sentence(sentence, max_length=2000)
            hindiPart = ''
            for i in range(len(small_sentences)):
                current_translate = GoogleTranslator(source='en', target='hi').translate(small_sentences[i])
                hindiPart = hindiPart + current_translate
        else:
            hindiPart = GoogleTranslator(source='en', target='hi').translate(sentence)
        hindiText=hindiText + hindiPart
        hindi.append(hindiText)
    return hindi

In [100]:
input_path="./data/summary/"
output_path="./data/summary_results/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

data = pd.read_csv(f'{input_path}data.csv')
data = data.head(2)
sentences = data['sentences'].apply(eval)
sentences_english = data['sentences_english'].apply(eval)
tokens = data['tokens'].apply(eval)
tokens_english = data['english_tokens'].apply(eval)

summary=[]
summary_english=[]
for i in tqdm(range(len(sentences))):
    summary.append(summarize_lexRank( sentences[i] ,tokens[i]))
    summary_english.append(convertToHindi(summarize_lexRank(  sentences_english[i] ,tokens_english[i])))

data['summary'] = summary
data['summary_english'] = summary_english

data.to_csv(f'{output_path}lexrank.csv', index=False)

  0%|          | 0/2 [00:00<?, ?it/s]